In [55]:
# Import libraries.
%pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from keras import layers
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

ERROR! Session/line number was not unique in database. History logging moved to new session 10
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [56]:

# load and visualize the data
dataset = pd.read_csv('insurance.csv')
dataset.tail()

,age,sex,bmi,children,smoker,region,expenses
1333,50,male,31.0,3,no,northwest,10600.55
1334,18,female,31.9,0,no,northeast,2205.98
1335,18,female,36.9,0,no,southeast,1629.83
1336,21,female,25.8,0,no,southwest,2007.95
1337,61,female,29.1,0,yes,northwest,29141.36


In [57]:
# split data for training and testing
y = dataset.expenses
data_features = ['age', 'sex', 'bmi', 'children', 'smoker', 'region']
X = dataset[data_features]

train_X, val_X, train_y, val_y = train_test_split(X, y, train_size = 0.8, test_size = 0.2, random_state = 0)

In [58]:
# check for missing values
cols_missing = [col for col in train_X.columns if train_X[col].isnull().any()]
print(cols_missing)

[]


In [60]:
# check for categorical values
categ = (train_X.dtypes == 'object')
categ_cols = list(categ[categ].index)
print("Columns with categorical values are:")
print(categ_cols)

Columns with categorical values are:
['sex', 'smoker', 'region']


In [62]:
# process categorical values
from sklearn.preprocessing import OneHotEncoder

my_encoder = OneHotEncoder(handle_unknown='ignore', sparse = False)

OH_cols_train = pd.DataFrame(my_encoder.fit_transform(train_X[categ_cols]))
OH_cols_val = pd.DataFrame(my_encoder.transform(val_X[categ_cols]))

OH_cols_train.index = train_X.index
OH_cols_val.index = val_X.index

num_train_X = train_X.drop(categ_cols, axis = 1)
num_val_X = val_X.drop(categ_cols, axis = 1)


OH_train_X = pd.concat([num_train_X, OH_cols_train], axis = 1)
OH_val_X = pd.concat([num_val_X, OH_cols_val], axis = 1)

OH_train_X.columns = OH_train_X.columns.astype(str)
OH_val_X.columns = OH_val_X.columns.astype(str)

# OH_train_X.rename(columns = {'3' : 'smoker'}, inplace = True)
# OH_val_X.rename(columns = {'3' : 'smoker'}, inplace = True)

# OH_train_X.rename(columns = {'1' : 'sex'}, inplace = True)
# OH_val_X.rename(columns = {'1' : 'sex'}, inplace = True)

print(OH_val_X)
print(val_X)



      age   bmi  children    0    1    2    3    4    5    6    7
578    52  30.2         1  0.0  1.0  1.0  0.0  0.0  0.0  0.0  1.0
610    47  29.4         1  1.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0
569    48  40.6         2  0.0  1.0  0.0  1.0  0.0  1.0  0.0  0.0
1034   61  38.4         0  0.0  1.0  1.0  0.0  0.0  1.0  0.0  0.0
198    51  18.1         0  1.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0
...   ...   ...       ...  ...  ...  ...  ...  ...  ...  ...  ...
1084   62  30.5         2  1.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0
726    41  28.4         1  0.0  1.0  1.0  0.0  0.0  1.0  0.0  0.0
1132   57  40.3         0  0.0  1.0  1.0  0.0  1.0  0.0  0.0  0.0
725    30  39.1         3  1.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0
963    46  24.8         3  0.0  1.0  1.0  0.0  1.0  0.0  0.0  0.0

[268 rows x 11 columns]
      age     sex   bmi  children smoker     region
578    52    male  30.2         1     no  southwest
610    47  female  29.4         1     no  southeast
569    48    male  40.6    

c:\Users\michels\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [63]:
my_model= LinearRegression()
my_model.fit(OH_train_X, train_y)
predict = my_model.predict(OH_val_X)
score = mean_absolute_error(val_y, predict)
print(score)



3933.3062578580643
